In [104]:
from pathlib import Path
from tqdm.notebook import tqdm
from tabulate import tabulate
import random
import pycountry
# import networkx as nx
from dateutil import parser
import json
from datetime import datetime
from collections import Counter
# from langdetect import detect
from omnibelt import load_json, save_json

In [2]:
localroot = Path().absolute().parent
localroot

PosixPath('/home/fleeb/workspace/code/nnn')

In [3]:
# root = Path('/Users/homeworld/workspace/local_data/nnn')
root = Path('/home/fleeb/workspace/local_data/nnn')
print(list(p.name for p in root.glob('*')))

['raw_news', 'ftlang.json', 'full.json', 'prompts', 'langdetect.json', 'dense', 'temp', 'global-news-headlines', 'global-news-headlines.zip', 'assets']


In [108]:
recs = (root/'global-news-headlines').glob('**/*.json')
recs = list(recs)
len(recs)

54

In [4]:
articles = []
for rec in tqdm(recs):
	articles.extend(load_json(rec))
len(articles)

  0%|          | 0/54 [00:00<?, ?it/s]

4719199

In [131]:
codes = [rec.stem.split('-')[-1] for rec in recs]
countries = {code: pycountry.countries.get(alpha_2=code.upper()).name.split(',')[0] for code in codes}
print(tabulate(sorted(countries.items())))

--  --------------------
ae  United Arab Emirates
ar  Argentina
at  Austria
au  Australia
be  Belgium
bg  Bulgaria
br  Brazil
ca  Canada
ch  Switzerland
cn  China
co  Colombia
cu  Cuba
cz  Czechia
de  Germany
eg  Egypt
fr  France
gb  United Kingdom
gr  Greece
hk  Hong Kong
hu  Hungary
id  Indonesia
ie  Ireland
il  Israel
in  India
it  Italy
jp  Japan
kr  Korea
lt  Lithuania
lv  Latvia
ma  Morocco
mx  Mexico
my  Malaysia
ng  Nigeria
nl  Netherlands
no  Norway
nz  New Zealand
ph  Philippines
pl  Poland
pt  Portugal
ro  Romania
rs  Serbia
ru  Russian Federation
sa  Saudi Arabia
se  Sweden
sg  Singapore
si  Slovenia
sk  Slovakia
th  Thailand
tr  Turkey
tw  Taiwan
ua  Ukraine
us  United States
ve  Venezuela
za  South Africa
--  --------------------


In [130]:
print(countries)

{'gb': 'United Kingdom', 'ar': 'Argentina', 'pl': 'Poland', 'sk': 'Slovakia', 'us': 'United States', 'eg': 'Egypt', 'no': 'Norway', 'ph': 'Philippines', 'at': 'Austria', 'rs': 'Serbia', 'tw': 'Taiwan', 'be': 'Belgium', 'cu': 'Cuba', 'sa': 'Saudi Arabia', 'th': 'Thailand', 'id': 'Indonesia', 'ru': 'Russian Federation', 'ch': 'Switzerland', 'fr': 'France', 'lt': 'Lithuania', 'tr': 'Turkey', 'de': 'Germany', 'cz': 'Czechia', 'pt': 'Portugal', 'ae': 'United Arab Emirates', 'it': 'Italy', 'cn': 'China', 'lv': 'Latvia', 'nl': 'Netherlands', 'hk': 'Hong Kong', 'ca': 'Canada', 'br': 'Brazil', 'hu': 'Hungary', 'kr': 'Korea', 'si': 'Slovenia', 'au': 'Australia', 'my': 'Malaysia', 'ie': 'Ireland', 'ua': 'Ukraine', 'in': 'India', 'ma': 'Morocco', 'bg': 'Bulgaria', 'ng': 'Nigeria', 'il': 'Israel', 'se': 'Sweden', 'za': 'South Africa', 've': 'Venezuela', 'nz': 'New Zealand', 'jp': 'Japan', 'sg': 'Singapore', 'gr': 'Greece', 'mx': 'Mexico', 'co': 'Colombia', 'ro': 'Romania'}


In [132]:
sel = {c: n for c,n  in countries.items() if c not in {'sa', 'ie', 'sg', 'us', 'ph', 'au', 'my', 'za', 'in', 'nz', 'ca', 'gb', 'ng'}}
len(sel)

41

In [134]:
print(' '.join(sorted(sel)))

ae ar at be bg br ch cn co cu cz de eg fr gr hk hu id il it jp kr lt lv ma mx nl no pl pt ro rs ru se si sk th tr tw ua ve


In [106]:
# by_loc = {}
# for article in tqdm(articles):
# 	for instance in article['instances']:
# 		by_loc.setdefault(instance['location'], []).append(article)
country_names = {code: pycountry.countries.get(alpha_2=code.upper()).name.split(',')[0] for code in by_loc}
clusters = {
	'english': {'sa', 'ie', 'sg', 'us', 'ph', 'au', 'my', 'za', 'in', 'nz', 'ca', 'gb', 'ng'}, 
	'spanish': {'ar', 've', 'co', 'cu', 'mx'}, 
	'arabic': {'ae', 'eg'}, 
	'german': {'at', 'de', 'ch'}, 
	'chinese': {'tw', 'hk', 'cn'}, 
	'french': {'be', 'fr', 'ma'}, 
	'portuguese': {'pt', 'br'},
}
# to_cluster = {country_names[loc]: cluster for cluster, locs in clusters.items() for loc in locs}
len(by_loc)

  0%|          | 0/82324 [00:00<?, ?it/s]

3

--  ---------
hk  Hong Kong
tw  Taiwan
cn  China
--  ---------


In [ ]:
def show_date(date):
	return date.strftime('%d %b%y')
def get_locs(article):
	return [f'{country_names[loc]}' for loc in sorted(set(i['location'] for i in article['instances']))]
def get_cats(article):
	return [f'<{cat}>' for cat in sorted(set(i['category'] for i in article['instances']))]
def view_article(art, detailed=False):
	cats = ' '.join(get_cats(art))
	locs = ', '.join(map(repr,get_locs(art)))
	if 'published' not in art:
		art['published'] = parser.parse(art['publishedAt'])
	for i in art['instances']:
		if 'collected' not in i:
			i['collected'] = parser.parse(i['collectedAt'])
	first = min(i['collected'] for i in art['instances'])
	last = max(i['collected'] for i in art['instances'])
	timing = f'{show_date(first)}' if first==last else f'{show_date(first)} - {show_date(last)}'
	print(f'''Title: {art['title']!r}
Categories: {cats}   ---   {locs} ({timing})
Published: {show_date(art['published'])}'''
	) # ({art['source-name']})
	if detailed:
		print(f'''Description: {art['description']}''')
		# print(f'''Content: {art['content']}''')

In [9]:
import json
from omnibelt import load_json

In [6]:
articles = load_json(root / 'global-news-headlines' / f'news-headlines-{loc}.json')
len(articles)

82324

In [7]:
unique = {}
for article in tqdm(articles):
	unique.setdefault((article['title'], article.get('description')), []).append(article['ID'])
len(unique)

  0%|          | 0/82324 [00:00<?, ?it/s]

80449

In [10]:
lines = [json.dumps({'title': title, 'description': desc, 'ids': ids}) for (title, desc), ids in unique.items()]
(localroot/'interface'/'prompts'/f'prompts-{loc}.jsonl').write_text('\n'.join(lines));

In [121]:
def load_jsonl(path):
	return [json.loads(line) for line in path.read_text().split('\n') if len(line)]
def save_jsonl(path, records):
	path.write_text('\n'.join(json.dumps(line) for line in records))
def save_jsonl_dict(path, data):
	save_jsonl(path, [{'title': title, 'description': desc, 'ids': ids} for (title, desc), ids in data.items()])
def to_ids(records):
	ids, errs = {}, set()
	for record in records:
		payload = record['title'], record.get('description')
		for ID in record.get('ids', [record.get('ID')]):
			if ID in ids:
				errs.add(ID)
			ids[ID] = payload
	return ids, errs
def group_ids(id_dict):
	table = {}
	for ID, value in id_dict.items():
		table.setdefault(value, []).append(ID)
	return table
def find_matches(ids1, ids2, strict=True):
	match, nomatch = set(), set()
	for ID in set(ids1) & set(ids2):
		(match if (ids1[ID] == ids2[ID]
				   if strict
				   else any(term1 is not None and len(term1) and term1 == term2 for term1, term2 in zip(ids1[ID], ids2[ID])))
		 else nomatch).add(ID)
	return match, nomatch
def remove_ids(id_dict, ids):
	for ID in ids:
		if ID in id_dict:
			id_dict.pop(ID)
	# return id_dict


In [ ]:
loc = 'co'

In [114]:
blobroot = Path('/home/fleeb/Downloads')
blobpaths = list(blobroot.glob('saved_records_*.jsonl'))

promptpath = localroot/'interface'/'prompts'/f'prompts-{loc}.jsonl'
assert promptpath.exists()

completedpath = localroot / 'interface' / 'completed' / f'completed_{loc}.jsonl'
if not completedpath.exists():
	completedpath.touch()

len(blobpaths)

0

In [74]:
new = [rec for blobpath in tqdm(blobpaths) for rec in load_jsonl(blobpath)]
print(f'Loaded {len(new)} new candidates.')

0it [00:00, ?it/s]

Loaded 0 new records


In [123]:
completed = load_jsonl(completedpath)
complids, errs = to_ids(completed)
assert not errs
len(completed)

194847

In [ ]:
candids, errs = to_ids(new)
assert not errs
len(candids)

In [93]:
dupes, inconsistent = find_matches(candids, complids)
print(f'Found {len(dupes)} duplicates (ignoring).')
if len(inconsistent):
	print(f'Found {len(inconsistent)} inconsistent records!')
complids.update(candids)

Found 0 duplicates (ignoring).
Found 0 inconsistent records!


In [124]:
todo = load_jsonl(promptpath)
todoids, errs = to_ids(todo)
assert not errs
len(todo)

89394

In [128]:
foreign = {ID: val for ID, val in complids.items() if ID not in todoids}
original = {ID: val for ID, val in complids.items() if ID in todoids}
len(foreign), len(original)

(106458, 91184)

In [127]:
save_jsonl_dict(localroot / 'interface' / 'completed' / f'completed_cu.jsonl', group_ids(foreign))

In [129]:
save_jsonl_dict(localroot / 'interface' / 'completed' / f'completed_co.jsonl', group_ids(original))

In [94]:
incomplete, unnecessary = find_matches(complids, todoids)
print(f'Found {len(incomplete)} untranslated prompts in completed.')
print(f'Found {len(unnecessary)} unnecessary prompts.')

Found 0 untranslated prompts in completed.
Found 0 unnecessary prompts.


In [98]:
remove_ids(todoids, unnecessary)
remove_ids(complids, incomplete)

In [99]:
save_jsonl_dict(promptpath, group_ids(todoids));
save_jsonl_dict(completedpath, group_ids(complids));

In [103]:
None == None

True

In [ ]:
for blobpath in blobpaths:
	blobpath.unlink()

In [52]:
with completedpath.open('a+') as dst:
	for blobpath in tqdm(blobpaths):
		with blobpath.open('r') as src:
			dst.write(src.read())
			dst.write('\n')
		blobpath.unlink()

  0%|          | 0/20 [00:00<?, ?it/s]

In [53]:
# remove completed
# done = [json.loads(line) for line in tqdm(completedpath.read_text().split('\n')) if len(line)]
done = []
for line in tqdm(completedpath.read_text().split('\n')):
	try:
		if len(line):
			done.append(json.loads(line))
	except json.JSONDecodeError:
		raw1, raw2 = line.split('}{')
		done.append(json.loads(raw1+'}'))
		done.append(json.loads('{'+raw2))
len(done)

  0%|          | 0/10263 [00:00<?, ?it/s]

10300

In [54]:
promptpath = localroot/'interface'/'prompts'/f'prompts-{loc}.jsonl'
prompts = [json.loads(line) for line in tqdm(promptpath.read_text().split('\n'))]
len(prompts)

  0%|          | 0/72149 [00:00<?, ?it/s]

72149

In [69]:
doneids = {ID: (prompt['headline'], prompt.get('description')) for prompt in done for ID in prompt['ID']}
todoids = {ID: (prompt['title'], prompt.get('description')) for prompt in prompts for ID in prompt['ids']}
overlap = set(doneids) & set(todoids)
print(f'Previously needed to complete {len(todoids)} IDs')
print(f'{len(doneids)} IDs have now been completed')
print(f'Found {len(overlap)} IDs that need to be resolved')
# len(doneids), len(todoids), len(overlap)

Previously needed to complete 73768 IDs
10616 IDs have now been completed
Found 2060 overlapping IDs


2060

In [63]:
good = set()
bad = set()
for check in overlap:
	(bad if doneids[check] == todoids[check] else good).add(check)
len(good), len(bad)

(1442, 618)

In [66]:
for g in good:
	if g in todoids:
		todoids.pop(g)
prompt_groups = {}
for ID, data in todoids.items():
	prompt_groups.setdefault(data, []).append(ID)
promptpath.write_text('\n'.join(json.dumps({'title': title, 'description': desc, 'ids': ids}) for (title, desc), ids in prompt_groups.items()));
len(prompt_groups)

70749

In [70]:
for b in bad:
	if b in doneids:
		doneids.pop(b)
done_groups = {}
for ID, data in doneids.items():
	done_groups.setdefault(data, []).append(ID)
completedpath.write_text('\n'.join(json.dumps({'title': title, 'description': desc, 'ids': ids}) for (title, desc), ids in done_groups.items()));
len(done_groups)

9692

In [68]:
len(prompt_groups) + len(done_groups)

80441

In [ ]:

# filter completed

In [57]:

cleaned = [prompt for prompt in done if not any(ID in todoids and doneids[ID] == prompt['title'] for ID in prompt['ids'])]
fixed = [prompt for prompt in prompts if not any(ID in doneids and doneids[ID] != prompt['title'] for ID in prompt['ids'])]
len(dupes), len(fixed)

(600, 70749)

In [50]:
promptpath.write_text('\n'.join(json.dumps(prompt) for prompt in fixed));

In [8]:
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<title>News Headlines</title>
	<style>
		body {{
			font-family: Arial, sans-serif;
			margin: 20px;
		}}
		.headline {{
			padding: 5px;
			border-bottom: 1px solid #ddd;
			display: flex;
			justify-content: space-between;
		}}
		.title {{
			flex: 2;
		}}
		.ids {{
			flex: 1;
			text-align: right;
		}}
		#saveButton {{
			position: absolute;
			top: 10px;
			left: 10px;
		}}
	</style>
</head>
<body>
	<button id="saveButton">Save & Next</button>
	<div id="news-container">
		{headlines}
	</div>
	<script>
		let currentIndex = 0;
		const N = {N};
		const headlines = document.querySelectorAll('.headline');

		// Initial display of N headlines
		for(let i = 0; i < headlines.length; i++) {{
			if (i < N) {{
				headlines[i].style.display = 'flex';
			}} else {{
				headlines[i].style.display = 'none';
			}}
		}}

		document.getElementById('saveButton').addEventListener('click', function() {{
			let textToSave = '';
			for (let i = currentIndex; i < currentIndex + N; i++) {{
				if (headlines[i]) {{
					textToSave += headlines[i].innerText + '\\n';
					headlines[i].style.display = 'none';
				}}
			}}
			currentIndex += N;
			for (let i = currentIndex; i < currentIndex + N; i++) {{
				if (headlines[i]) {{
					headlines[i].style.display = 'flex';
				}}
			}}
			
			// Simulate saving to a file
			const blob = new Blob([textToSave], {{type: 'text/plain'}});
			const a = document.createElement('a');
			a.href = URL.createObjectURL(blob);
			a.download = 'saved_headlines.txt';
			document.body.appendChild(a);
			a.click();
			document.body.removeChild(a);
		}});
	</script>
</body>
</html>
"""

In [16]:
def generate_html_with_js(headline_info, N=5):
    # Generate headlines
    headlines = ""
    for headline, ids in headline_info:
        ids_str = ', '.join(map(str, ids))
        headlines += f'<div class="headline"><span class="title">{headline}</span><span class="ids">[{ids_str}]</span></div>\n'
    
    # Insert headlines and N into the HTML template
    return html_template.format(headlines=headlines, N=N)

# Generate the HTML content with JavaScript functionality
# html_content_with_js = generate_html_with_js(sample_headlines_dict)
# html_content_with_js[:1000]  # Displaying only the first 1000 characters for brevity


In [23]:
import json

In [ ]:
lines = [json.dumps({'title': title, 'description': desc, 'ids': ids}) for (title, desc), ids in unique.items()]


In [12]:
full = list(unique.items())

In [21]:
(root / 'prompts')

In [17]:
html_content_with_js = generate_html_with_js(full[:100])

In [ ]:
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<title>News Headlines</title>
	<style>
		body {{
			font-family: Arial, sans-serif;
			margin: 20px;
		}}
		.headline {{
			padding: 5px;
			border-bottom: 1px solid #ddd;
			display: flex;
			justify-content: space-between;
		}}
		.title {{
			flex: 2;
		}}
		.ids {{
			flex: 1;
			text-align: right;
		}}
	</style>
</head>
<body>
	<div id="news-container">
		{headlines}
	</div>
</body>
</html>
"""
def generate_html_from_dict(headlines_dict):
    # Base HTML template
    # Generate headlines
    headlines = ""
    for headline, ids in headlines_dict.items():
        ids_str = ', '.join(map(str, ids))
        headlines += f'<div class="headline"><span class="title">{headline}</span><span class="ids">[{ids_str}]</span></div>\n'
    
    # Insert headlines into the HTML template
    return html_template.format(headlines=headlines)

# Sample headlines dictionary for testing
sample_headlines_dict = {
    "Breaking: Earthquake in Paris": [101, 102, 103],
    "Rain causes uproar in London": [104, 105],
    "London witnesses Earthquake phenomenon": [106]
}

# Generate the HTML content
html_content_from_dict = generate_html_from_dict(sample_headlines_dict)
html_content_from_dict[:1000]  # Displaying only the first 1000 characters for brevity


In [ ]:
def generate_html_with_js_and_file_io(N=5):
    # Base HTML template with file input and JavaScript for file handling

    
    return html_template.format(N=N)

# Generate the HTML content with JavaScript functionality for file I/O
html_content_with_file_io = generate_html_with_js_and_file_io()
html_content_with_file_io[:1000]  # Displaying only the first 1000 characters for brevity
